In [1]:
from scipy.integrate import odeint

In [ ]:
# plasma parameters

#we need
plasma_params = {'n_i': 1e16, 'T_e': 1, 'T_i': 1, 'm_i': 1}

## Calculate plasma parameters

Zimmermann's model is described by the the the system of ODEs

In [ ]:
def zimmerman_func(x, params):


In [ ]:
odeint